# 0. Import Data

In [1]:
# ! pip install tensorflow

In [2]:
# ! pip install scikit-learn

In [10]:
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [1]:
! pip install gdown

  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
Using cached filelock-3.16.1-py3-none-any.whl (16 kB)


In [ ]:
gdrive_url = ""

In [11]:
import pickle 

with open(r"C:\Users\bened\DataScience\ANLP\AT2\preprocessed_data\processed_screenplays_final.pkl", "rb") as f:
    data = pickle.load(f)

In [12]:
import pandas as pd 

root_path = r'C:\\Users\bened\DataScience\ANLP\AT2'

meta_df = pd.read_csv(f'{root_path}\\movie_meta_data.csv')
meta_df.head()

,imdbid,title,akas,year,metascore,imdb user rating,number of imdb user votes,awards,opening weekend,producers,...,casting directors,cast,countries,age restrict,plot,plot outline,keywords,genres,taglines,synopsis
0,120770,A Night at the Roxbury,"Une nuit au Roxbury (France), Movida en el Rox...",1998,26,6,56537,NaN,United States:,"Marie Cantin, Erin Fraser, Amy Heckerling, Ste...",...,Jeff Greenberg,"Will Ferrell, Chris Kattan, Raquel Gardner, Vi...",United States,"Argentina:13, Australia:M, Brazil:14, Canada:P...",Two dim-witted brothers dream of owning their ...,"The Roxbury Guys, Steve and Doug Butabi, want ...","woman-on-top, nightclub, car-accident, 1990s, ...","Comedy, Music, Romance",Score!,NaN
1,132512,At First Sight,"Sight Unseen (United States), Premier regard (...",1999,40,6,12922,NaN,United States:,"Rob Cowan, Roger Paradiso, Irwin Winkler",...,"Kerry Barden, Billy Hopkins, Suzanne Smith","Val Kilmer, Mira Sorvino, Kelly McGillis, Stev...",United States,"Argentina:13, Australia:M, Canada:PG::(Alberta...",A blind man has an operation to regain his sig...,First Sight is true to the title from start to...,"visual-agnosia, brother-sister-relationship, r...","Drama, Romance","Only Love Can Bring You To Your Senses., Scien...",NaN
2,118661,The Avengers,"Chapeau melon et bottes de cuir (France), Mit ...",1998,12,3,40784,"FMCJ Award 1998, Golden Reel Award 1999, Razzi...","United States: $10,305,957, 16 Aug 1998","Susan Ekins, Jerry Weintraub",...,Susie Figgis,"Ralph Fiennes, Uma Thurman, Sean Connery, Patr...",United States,"Argentina:13, Australia:PG, Brazil:10, Canada:...",Two British Agents team up to stop Sir August ...,"British Ministry Agent John Steed, under direc...","good-versus-evil, heroine, evil-man, villain, ...","Action, Adventure, Sci-Fi, Thriller","Mrs. Peel, we're needed., Extraordinary crimes...",NaN
3,215545,Bamboozled,"The Very Black Show (France), It's Showtime (G...",2000,54,6,10373,"Golden Berlin Bear 2001, Black Reel 2001, Imag...",United States:,"Jon Kilik, Spike Lee, Kisha Imani Cameron",...,Aisha Coley,"Damon Wayans, Savion Glover, Jada Pinkett Smit...",United States,"Australia:MA, Finland:K-15, France:Tous public...",A frustrated African-American TV writer propos...,"Dark, biting satire of the television industry...","television-industry, african-american, referen...","Comedy, Drama, Music",Starring the great negroe actors,"In a New York City residence, Pierre Delacroix..."
4,118715,The Big Lebowski,"El gran Lebowski (Spain), O Grande Lebowski (P...",1998,71,8,724388,"Honorable Mention 1998, ACCA 1998, Golden Berl...","United States: $5,533,844, 08 Mar 1998","Tim Bevan, John Cameron, Ethan Coen, Eric Fell...",...,John S. Lyons,"Jeff Bridges, John Goodman, Julianne Moore, St...","United States, United Kingdom","Argentina:16, Argentina:18::(cable rating), Au...","Jeff ""The Dude"" Lebowski, mistaken for a milli...","When ""the dude"" Lebowski is mistaken for a mil...","rug, nihilism, pornographer, bowling-alley, de...","Comedy, Crime, Sport",Hay quienes tratan de ganarse la vida sin move...,A tumbleweed rolls up a hillside just outside ...


# Phrases

In [6]:
import numpy as np
import re

def preprocess_for_phrases(data):

    labeled_sentences = {}

    for screenplay_idx, tokens in data.items():

        current_sentence = []
        processed_line = []
        line_label = None

        for token in tokens:
            
            # parse label
            if token.startswith('@'):
                line_label = np.int8(token[1:])
                labeled_sentences[line_label] = []

            # skip if token contains non-word chars 
            elif re.search(r'\W', token) and token not in {'.', '\n'}:
                continue

            elif token == ':':
                continue

            # skip over '-' 
            elif token == '-':
                continue
            
            elif token == '.':
                if current_sentence:
                    processed_line.append(current_sentence)
                    current_sentence = []
            
            elif token == '\n':
                if current_sentence:
                    processed_line.append(current_sentence)
                    current_sentence = []
            
            else:
                current_sentence.append(token)
        
        if current_sentence:
            processed_line.append(current_sentence)
        
        if line_label is not None:
            labeled_sentences[line_label].extend(processed_line)
        
    return labeled_sentences


In [7]:
print(len(data))

1995


In [8]:
print(data[0][:10])

['@1', 'roxbury', '\n ', 'write', 'steve', 'koren', 'ferrell', 'chris', 'kattan', 'june']


In [9]:
def back_to_sents(dict):

    sentences_dict = {}
    
    for key, value in dict.items():
        # list of sentences of each value
        sentences = []
        current_sentence = []

        for t in value:
            if t.strip() == '\n':
                if current_sentence:
                    sentences.append(current_sentence)
                    current_sentence = []
            else:
                current_sentence.append(t)
            
        if current_sentence:
            sentences.append(current_sentence)

        sentences_dict[key] = sentences
    
    return sentences_dict



In [10]:
sents_data = back_to_sents(data)

In [11]:
def prep_phrases(sents_data):
    sentences =  []
    for key, sent_list in sents_data.items():
        sentences.extend(sent_list)
    return sentences 

sents = prep_phrases(sents_data)

In [13]:
from gensim.models import phrases

In [14]:
bigrams = phrases.Phrases(
    sentences=sents,
    connector_words=phrases.ENGLISH_CONNECTOR_WORDS
)
bigrams_dict = bigrams.export_phrases()
bigrams_df = pd.DataFrame.from_dict(
    bigrams_dict, 
    orient='index'
)
bigrams_df.reset_index(inplace=True)
bigrams_df.columns = ["phrase", "score"]
bigrams_df.sort_values('score', inplace=True, ascending=False)

In [15]:
bigrams_df

,phrase,score
34782,there's_something_about_mary_p.,483308.155063
33870,window.addeventlistener_domcontentloade,353530.965278
31034,snacky_smores,303026.541667
19419,bombosity_knickety,296842.326531
19420,knickety_knackety,296842.326531
...,...,...
19396,hesitates_moment,10.004623
24882,bathtub_fill,10.003902
5267,street_singer,10.003385
35025,run_upstage,10.002196


In [17]:
bigram_model = bigrams.freeze()
bigram_model.save(r'C:\Users\bened\DataScience\ANLP\AT2\models\bigram_model2.pkl')

In [21]:
# threshold to 25

bigrams_select = phrases.Phrases(
    sentences=sents,
    connector_words=phrases.ENGLISH_CONNECTOR_WORDS,
    threshold=10,
    min_count=10
)
bigrams_dict = bigrams_select.export_phrases()
bigrams_df = pd.DataFrame.from_dict(
    bigrams_dict, 
    orient='index'
)
bigrams_df.reset_index(inplace=True)
bigrams_df.columns = ["phrase", "score"]
bigrams_df.sort_values('score', inplace=True, ascending=False)

In [23]:
bigrams_df['score'] = bigrams_df['score'].round(4)
print(bigrams_df.head())
print(bigrams_df.tail()) 

                                phrase        score
16421  there's_something_about_mary_p.  464129.2600
15565                      zoobi_doobi  166367.5131
16105                    durbar_court/  151513.2708
11446                        kol_nidre  141674.7468
11457                       yom_kippur  131376.6684
              phrase    score
4890     cellar_door  10.0050
5815     poise_ready  10.0038
6515     inches_long  10.0033
11273  cloud_exhaust  10.0031
11076     pile_trash  10.0002


In [24]:
bigrams_df

,phrase,score
16421,there's_something_about_mary_p.,464129.2600
15565,zoobi_doobi,166367.5131
16105,durbar_court/,151513.2708
11446,kol_nidre,141674.7468
11457,yom_kippur,131376.6684
...,...,...
4890,cellar_door,10.0050
5815,poise_ready,10.0038
6515,inches_long,10.0033
11273,cloud_exhaust,10.0031


In [25]:
bigrams_table = pd.concat([bigrams_df.head(10), bigrams_df.tail(10)])
bigrams_table.to_csv(r"C:\Users\bened\DataScience\ANLP\AT2\visualizations\bigrams_table.csv")


In [ ]:
for phrase, score in bigrams_df.iterrows():
    print(phrase, ",", score)

In [20]:
bigrams_df.iloc[3:13]

,phrase,score
9545,kama_sutra,227269.906250
18712,deutsches_technikmuseum,223773.446154
11287,khor_kalba,222631.744898
13159,bryn_mawr,220382.939394
9476,kol_nidre,204641.300866
15128,snacky_smores,202017.694444
16583,durbar_court/,202017.694444
14508,piggly_wiggly,202017.694444
4962,belo_quinto,194802.776786
12349,kuala_lumpur,191385.184211


In [33]:
trigrams = phrases.Phrases(bigrams_select[sents], min_count=15, threshold=75)
# trigram_model = phrases.Phraser(trigrams)
trigrams_dict = trigrams.export_phrases()
trigrams_df = pd.DataFrame.from_dict(
    trigrams_dict, 
    orient='index'
)
trigrams_df.reset_index(inplace=True)
trigrams_df.columns = ["phrase", "score"]
trigrams_df['score'] = trigrams_df['score'].round(4)
trigrams_df.sort_values('score', inplace=True, ascending=False)
trigrams_df

,phrase,score
4502,geel_piet,7.425097e+07
4815,nai_nai,2.094580e+07
4332,chattar_lal,1.049702e+07
3655,yom_kippur,1.032060e+07
5080,ibn_khaldun,7.491990e+06
...,...,...
1853,central_intelligence_agency,7.519960e+01
298,hello_hello,7.518800e+01
1256,keen_interest,7.507000e+01
677,cough_fit,7.505850e+01


In [32]:
print(trigrams_df.tail(20))

                phrase    score
18283    tile_bathroom  10.0193
6718          cry_loud  10.0191
4145         thank_hon  10.0190
17281         wag_head  10.0170
504         hear_sound  10.0161
12481         eat_chip  10.0158
10161       duck_sight  10.0127
4321     couple_minute  10.0118
13044   hotel_employee  10.0103
6057         cook_food  10.0089
15900   unaware_danger  10.0081
1072      roll_thunder  10.0072
7325         watch_mtv  10.0064
16681  job_application  10.0063
6198      day_new_york  10.0046
10172        dump_load  10.0035
7006   absolutely_idea  10.0025
18569         pig_tail  10.0018
14930    sheer_curtain  10.0012
15842      red_ferrari  10.0010


In [78]:
frozen_trigrams = trigrams.freeze() 
trigrams.save(r"C:\Users\bened\DataScience\ANLP\AT2\models\trigrams_final.pkl")

In [ ]:
del trigrams

# Word2Vec

In [132]:
print(type(sents))
print(sents[0])

<class 'list'>
['jonathan', 'ames', 'goldenrod', 'revision', 'green', 'revision', 'yellow', 'revision', 'revision', 'blue', 'revision', 'shooting', 'close', 'adult', 'man', 'mouth', 'water', 'joe', 'gulps', 'water', 'fill', 'air', 'stretch', 'smooth', 'surface', 'mist', 'droplet', 'moisture', 'form', 'break', 'miniature', 'rivulet', 'whisper', 'boy', 'voice', 'young', 'joe', 'count', 'sharp', 'inhale', 'clear', 'plastic', 'invert', 'face', 'joe', 'bag', 'suddenly', 'suck', 'hard', 'skin', 'distort', 'feature', 'inhuman', 'mask', 'boy', 'count', 'suspend', 'faint', 'beat', 'pulse', 'rise', 'replace', 'man', 'bag', 'balloon', 'cloud', 'man', 'breathe', 'slowly', 'plastic', 'bag', 'balloon', 'invert', 'boy', 'whisper', 'resume', 'man', 'never', 'blink', 'plastic', 'eye', 'unseee', 'rise', 'fall', 'mouth', 'nose', 'sign', 'life', 'titles', 'montage', 'glimpse', 'methodical', 'process', 'camera', 'tight', 'sound', 'heighten', 'fill', 'frame', 'passport', 'style', 'photograph', '16', 'years'

In [142]:
print(len(sents[0]))

6862


In [40]:
print(len(sents))

1995


In [35]:
from gensim.models import Word2Vec

In [36]:
cbow = Word2Vec(
    sentences=trigrams[sents],
    vector_size=64,
    window=8,
    sg=0
)

In [37]:
word_vectors = cbow.wv

In [80]:
cbow.save(r"C:\Users\bened\DataScience\ANLP\AT2\models\final_cbow.model")

In [81]:
del cbow

In [38]:
word_vectors.save(r"C:\Users\bened\DataScience\ANLP\AT2\models\final_wv.wordvectors")

In [41]:
import random

vocab = list(word_vectors.key_to_index.keys())
print(random.sample(vocab, 10))

['belted', 'morlocks', '11_-', 'yell', 'candlelit', 'snapshots', 'startlingly', 'hyde_park', 'eyeshade', 'machine_guns']


In [141]:
print(len(vocab))

312


In [61]:
print(len(sents))

1995


In [62]:
print(type(sents))

<class 'list'>


now need to attach the original keys to sents. 

In [42]:
def find_synonyms(vectors, word, topn=5):
    synonyms = vectors.most_similar(word, topn=topn)
    print(f"Words most similar to {word}:")
    for w, s in synonyms:
        print(f'{w}: {s:.4f}')

find_synonyms(word_vectors, 'fuck')

Words most similar to fuck:
fucking: 0.9588
asshole: 0.9057
shit: 0.8898
bitch: 0.8775
fuckin: 0.8614


In [43]:
find_synonyms(word_vectors, 'sex', 5)

Words most similar to sex:
masturbate: 0.7412
lesbian: 0.7181
girlfriend: 0.6957
fantasize: 0.6926
sexual: 0.6868


In [44]:
find_synonyms(word_vectors, 'cocaine', 5)

Words most similar to cocaine:
heroin: 0.8568
coke: 0.7380
drug: 0.7149
packet: 0.7128
vitamin: 0.7116


In [45]:
find_synonyms(word_vectors, 'naked', 5)

Words most similar to naked:
nude: 0.8235
wrap_towel: 0.6653
topless: 0.6474
underwear: 0.6417
shirtless: 0.6343


# Feature and Target Engineering

Encode Target (Aus ratings)

In [71]:
meta_df['age restrict'].head()

0    Argentina:13, Australia:M, Brazil:14, Canada:P...
1    Argentina:13, Australia:M, Canada:PG::(Alberta...
2    Argentina:13, Australia:PG, Brazil:10, Canada:...
3    Australia:MA, Finland:K-15, France:Tous public...
4    Argentina:16, Argentina:18::(cable rating), Au...
Name: age restrict, dtype: object

In [13]:
import re

def extract_aus_rating(string):
    pattern = re.compile(r'(?:^|, )Australia:([A-Z]{1,3}\d{0,2}\+?)[,]|$')
    match = re.search(pattern, string)
    if match:
        return match.group(1)
    else:
        return pd.NA

# drop rows with NA for age restrict 
meta_df = meta_df.dropna(how='any', subset='age restrict')

aus_classifications = meta_df['age restrict'].apply(extract_aus_rating)
print(aus_classifications.value_counts())

age restrict
M        994
MA15+    392
PG       348
MA       195
R        128
G        108
A         39
R18+      36
SOA       18
NRC       10
RC         1
Name: count, dtype: int64


From Wiki: 

Early film classification
The Commonwealth Film Censorship Board was created in 1917 to view, classify, and censor films imported from overseas. In the early years of the system there were 3 ratings:[4]

G for "general audiences"
A for "not suitable for children"
SOA for "suitable for adults only"
All ratings were advisory in nature and while distributors were required to display them on advertising, there were no restrictions on children's attendance. As such, films with adult ratings were still routinely censored.

A almost certainly corresponds to G. 
NRC is effectively PG. 
SOA is more ambiguous, we'll drop this. 
Extreme imbalance for 'RC': only one sample means it can't be represented in both training and testing data, so can't be used. 


In [14]:
aus_target_map = {
    'G': 0,
    'A': 0,
    'PG': 1,
    'NRC': 1,
    'M': 2,
    'MA': 3,
    'MA15+': 3,
    'R': 4,
    'R18+': 4
}

y = aus_classifications.map(aus_target_map)
y.value_counts()

age restrict
2.0    994
3.0    587
1.0    358
4.0    164
0.0    147
Name: count, dtype: int64

In [15]:
y = y.dropna()
print(y.shape)

(2250,)


In [16]:
y = y.astype(int)

In [17]:
def compute_class_balance(y):
    classes = sorted(list(y.unique()))
    value_counts = y.value_counts()
    for c in classes:
        class_ratio = value_counts[c] / len(y) * 100
        print(f"{c} represents {class_ratio}% of target data")

In [18]:
compute_class_balance(y)

0 represents 6.533333333333332% of target data
1 represents 15.911111111111111% of target data
2 represents 44.17777777777778% of target data
3 represents 26.08888888888889% of target data
4 represents 7.28888888888889% of target data


There is a significant class imbalance with regard to movies rated M (3); the other imbalances are less significant.

In [19]:
keys = list(data.keys())
print(keys[:10])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [20]:
print(data[0][:20])

['@1', 'roxbury', '\n ', 'write', 'steve', 'koren', 'ferrell', 'chris', 'kattan', 'june', '\n ', '@1', 'hear', 'love', 'haddaway', 'night', 'fall', 'partytime', 'begin', 'superimpose']


In [21]:
print(len(data))

1995


TODO: 
- convert '@n' to n 
- mark sequence boundaries at '\n' 

## Data Split

In [22]:
# X = 'Year' + data matched on index labels 
year = meta_df['year']
data_series = pd.Series(data)
data_series.head()

0    [@1, roxbury, \n , write, steve, koren, ferrel...
1    [@0, first, gold, light, dapple, valley, seem,...
2    [1blue, clouds, 2ext, @1, flat, horizon, stret...
3    [lee, \n , voice, malcolm, \n , hoodwink, take...
4    [@0, float, steep, scrubby, slope, hear, male,...
dtype: object

## Index Alignment

In [23]:
# concatenate the three series together to ensure same indices 
df = pd.concat([year, data_series, y], axis=1, join='inner')
print(df.shape)

(1633, 3)


In [83]:
del year, data_series, data, meta_df

1633 observations is perfectly fine.

In [24]:
df.columns = ['year','screenplay','target']

In [25]:
X = df[['year', 'screenplay']]
y = df.pop('target')

In [26]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

In [60]:
X_train.to_csv(r"C:\Users\bened\DataScience\ANLP\AT2\36118_NLP_Spring\data_splits\X_train_2.csv")
X_test.to_csv(r"C:\Users\bened\DataScience\ANLP\AT2\36118_NLP_Spring\data_splits\X_test_2.csv")

y_train.to_csv(r"C:\Users\bened\DataScience\ANLP\AT2\36118_NLP_Spring\data_splits\y_train_2.csv")
y_test.to_csv(r"C:\Users\bened\DataScience\ANLP\AT2\36118_NLP_Spring\data_splits\y_test_2.csv")

In [23]:
del X, y

# 2. Feature Engineering

## 2.1 Build Vocab 

In [28]:
def create_vocab(data):
    vocab = set()
    for tokens in data:
        vocab.update(tokens)
    return {word: idx + 1 for idx, word in enumerate(vocab)}

train_vocab = create_vocab(X_train['screenplay'])
test_vocab = create_vocab(X_test['screenplay'])
vocab = {**train_vocab, **test_vocab}

## Phrases

In [106]:
from gensim.models import Word2Vec, phrases

In [107]:
X_train.head()

,year,screenplay
792,2009,"[idiot, fit, screenplay, fiem, screenplay, a-1..."
403,2011,"[screenplay, james, l., brooks, \n , @0, brisk..."
1842,1996,"[@0, boss, jonathan, goldstein, john, francis,..."
8,1999,"[@0, hannah, allen, \n , @1, last, credit, app..."
332,2011,"[@1, @0, open, city, series, brief, shot, defi..."


In [110]:
screenplays = X_train['screenplay'] + X_test['screenplay']
print(type(screenplays))

<class 'pandas.core.series.Series'>


In [112]:
print(screenplays[0][:10])

TypeError: 'float' object is not subscriptable

In [111]:
print(type(screenplays[792]))

<class 'float'>


In [108]:
import numpy as np
import re

def preprocess_for_phrases(data):

    labeled_sentences = {}

    for screenplay_idx, tokens in data.items():

        current_sentence = []
        processed_line = []
        line_label = None

        for token in tokens:
            
            # parse label
            if token.startswith('@'):
                line_label = np.int8(token[1:])
                labeled_sentences[line_label] = []

            # skip if token contains non-word chars 
            elif re.search(r'\W', token) and token not in {'.', '\n'}:
                continue

            elif token == ':':
                continue

            # skip over '-' 
            elif token == '-':
                continue
            
            elif token == '.':
                if current_sentence:
                    processed_line.append(current_sentence)
                    current_sentence = []
            
            elif token == '\n':
                if current_sentence:
                    processed_line.append(current_sentence)
                    current_sentence = []
            
            else:
                current_sentence.append(token)
        
        if current_sentence:
            processed_line.append(current_sentence)
        
        if line_label is not None:
            labeled_sentences[line_label].extend(processed_line)
        
    return labeled_sentences


## 2.1. Token:Integer Mapping

In [30]:
X_train['screenplay']

792     [idiot, fit, screenplay, fiem, screenplay, a-1...
403     [screenplay, james, l., brooks, \n , @0, brisk...
1842    [@0, boss, jonathan, goldstein, john, francis,...
8       [@0, hannah, allen, \n , @1, last, credit, app...
332     [@1, @0, open, city, series, brief, shot, defi...
                              ...                        
1782    [write, saulnier, \n , @0, draft-, white, \n ,...
388     [@1, house, corpse, revise, living, see, littl...
1702    [shawn, christensen, \n , @0, card, december, ...
1576    [write, silent, film, illustrate, musically, t...
1980    [@1, woman, drive, country, road, field, side,...
Name: screenplay, Length: 1306, dtype: object

In [31]:
def tokens_to_seqs(data, vocab):
    return [[vocab[token] for token in tokens] for tokens in data]

X_train_seq = tokens_to_seqs(X_train['screenplay'], vocab)
X_test_seq = tokens_to_seqs(X_test['screenplay'], vocab)

In [32]:
print(np.shape(X_train_seq))
print(np.shape(X_test_seq))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1306,) + inhomogeneous part.

In [97]:
print(type(X_train_seq))
print(type(X_test_seq))

<class 'list'>
<class 'list'>


In [98]:
print(X_train_seq[0][:10])

[51870, 6650, 62248, 101830, 62248, 119097, 59314, 27781, 128261, 69104]


In [89]:
# average sequence length 
import numpy as np

avg_seq_length_train = np.mean([len(seq) for seq in X_train_seq])
avg_seq_length_test = np.mean([len(seq) for seq in X_test_seq])

print("Train avg. sequence length:", avg_seq_length_train)
print("Test avg sequence length:", avg_seq_length_test)

Train avg. sequence length: 10227.97549770291
Test avg sequence length: 10007.085626911316


In [92]:
print(np.max([len(seq) for seq in X_train_seq]))
print(np.max([len(seq) for seq in X_test_seq]))

34025
21527


In [99]:
X_combined = X_train_seq + X_test_seq
seq_lengths = np.array([len(seq) for seq in X_combined])
se = np.mean(seq_lengths) + 2 * np.std(seq_lengths)
print(f"Sequence lengths fall below {se} with 95% confidence")


Sequence lengths fall below 15527.905686500992 with 95% confidence


In [100]:
max_seq=15000

truncate to 10k at start

In [101]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_padded = pad_sequences(X_train_seq, maxlen=max_seq, padding='post', truncating='pre')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_seq, padding='post', truncating='pre')

In [102]:
X_train_padded.shape

(1306, 15000)

In [103]:
del X_train_seq, X_test_seq

In [104]:
train_years = np.array(X_train['year'].values).reshape(-1,1)
test_years = np.array(X_test['year'].values).reshape(-1,1)

## GloVe

In [105]:
from numpy import asarray
from numpy import zeros

embeddings_dict = dict()

glove_path = r"C:\\Users\bened\DataScience\ANLP\AT2\\glove.6B.100d.txt"

glove_file = open(glove_path, encoding='utf-8')

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dims = asarray(records[1:], dtype='float32')
    embeddings_dict[word] = vector_dims

glove_file.close()


In [32]:
embedding_matrix = zeros((len(vocab), 100))
for word, index in vocab.items():
    vector = embeddings_dict.get(word)
    if vector is not None:
        embedding_matrix[index] = vector

print(embedding_matrix.shape)

(188894, 100)


In [33]:
del embeddings_dict

In [41]:
print(type(X_train_padded), X_train_padded.dtype)
print(type(X_train['year']), X_train['year'].dtype)
print(type(y_train), y_train.dtype)

<class 'numpy.ndarray'> int32
<class 'pandas.core.series.Series'> int64
<class 'pandas.core.series.Series'> int32


In [54]:
X_train_year = X_train['year'].values
y_train = y_train.values

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [59]:
X_test_year = X_test['year'].values.astype(np.float32)

In [48]:
print(X_train['year'].isna().sum())

0


# Word2Vec

In [65]:
def reattach_sents(data_dict, sents):
    return {key: sent_list for key, sent_list in zip(data_dict.keys(), sents)}

In [77]:
sents_dict = reattach_sents(data, sents)

In [82]:
del data, sents

In [83]:
del aus_classifications, bigrams_select

In [67]:
def vector_transform(sentences, phrase_model, word_vectors):
    vectorized_sents = []
    for sent in sentences:
        trigram_sent = trigrams[sent]
        vectorized_sent = []
        for word in trigram_sent:
            if word in word_vectors:
                vectorized_sent.append(word_vectors[word])
            else:
                vectorized_sent.append(np.zeros(word_vectors.vector_size))
        vectorized_sents.append(vectorized_sent)
    return vectorized_sents

In [ ]:
vectorized_data = {
        key: vector_transform(sentences, frozen_trigrams, word_vectors)
        for key, sentences in sents_dict.items()
    }

In [72]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_sentences(vectorized_sents, maxlen, vector_size):
    padded_sents = pad_sequences(
        vectorized_sents,
        maxlen=maxlen,
        dtype='float16',
        padding='post',
        truncating='pre',
        value=np.zeros(vector_size)
    )
    return padded_sents

In [ ]:
padded_data = {
    key: pad_sentences(vectorized_data[key], 15000, 64)
        for key in vectorized_data
}

In [70]:
def prepare_vectors(data, sents, phrase_model, word_vectors, maxlen):
    indexed_sents = reattach_sents(data, sents)
    vector_size = word_vectors.vector_size
    vectorized_data = {
        key: vector_transform(sentences, phrase_model, word_vectors)
        for key, sentences in indexed_sents.items()
    }
    padded_data = {
        key: pad_sentences(vectorized_data[key], maxlen, vector_size)
    for key in vectorized_data
    }
    return padded_data

In [75]:
del bigrams_dict, data_series, df, keys, meta_df, sents_data, trigrams_df, trigrams_dict, vocab, X, X_test, X_train, y, y_test, y_train, year

In [76]:
maxlen=15000
padded_vectors = prepare_vectors(data, sents, trigrams, word_vectors, maxlen)

MemoryError: Unable to allocate 31.8 GiB for an array with shape (8880, 15000, 64) and data type float32

# FF NN

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers

maxlen=15000

# input for text data 
text_input = layers.Input(shape=(maxlen,), name='screenplay_input')

# densely connected layer that tries to boil down feature space to six classifiable elements
dense = layers.Dense(6, activation='relu', name='classifiable_elements')(text_input)

def one_to_one(inputs):
    return layers.multiply(inputs)

impact_units = [layers.Dense(1, activation='relu', name=f"Impact_score_{i}")(layers.Lambda(lambda x: x[:,i:i+1])(dense)) for i in range(6)]

# concatenate 1:1 impact scores
impact_scores = layers.concatenate(impact_units)

# year 
year_input = layers.Input(shape=(1,),name='year_input')

# concatenate impact scores with year input
merged = layers.concatenate([impact_scores, year_input])

# output layer 
output = layers.Dense(5, activation='softmax', name='output_layer')(merged)

ff = Model(inputs=[text_input, year_input], outputs=output)

ff.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['f1_score'])

ff.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ screenplay_input    │ (None, 15000)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ classifiable_eleme… │ (None, 6)         │     90,006 │ screenplay_input… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 1)         │          0 │ classifiable_ele… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 1)         │          0 │ classifiable_ele… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 1)         │          0 │ classifiable_ele… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_3 (Lambda)   │ (None, 1)         │          0 │ classifiable_ele… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_4 (Lambda)   │ (None, 1)         │          0 │ classifiable_ele… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_5 (Lambda)   │ (None, 1)         │          0 │ classifiable_ele… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Impact_score_0      │ (None, 1)         │          2 │ lambda[0][0]      │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Impact_score_1      │ (None, 1)         │          2 │ lambda_1[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Impact_score_2      │ (None, 1)         │          2 │ lambda_2[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Impact_score_3      │ (None, 1)         │          2 │ lambda_3[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Impact_score_4      │ (None, 1)         │          2 │ lambda_4[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Impact_score_5      │ (None, 1)         │          2 │ lambda_5[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 6)         │          0 │ Impact_score_0[0… │
│ (Concatenate)       │                   │            │ Impact_score_1[0… │
│                     │                   │            │ Impact_score_2[0… │
│                     │                   │            │ Impact_score_3[0… │
│                     │                   │            │ Impact_score_4[0… │
│                     │                   │            │ Impact_score_5[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ year_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 7)         │          0 │ concatenate[0][0… │
│ (Concatenate)       │                   │            │ year_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 90,058 (351.79 KB)

 Trainable params: 90,058 (351.79 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
import numpy as np

print(np.shape(X_train_padded))
print(np.shape(X_train_year))
print(np.shape(y_train))
print(np.shape(X_test_padded))
print(np.shape(X_test_year))
print(np.shape(y_test))

NameError: name 'X_train_padded' is not defined

In [61]:
ff_history = ff.fit(
    [X_train_padded, X_train_year], 
    y_train,
    validation_data=([X_test_padded, X_test_year], y_test),
    epochs=8, 
    batch_size=256)

Epoch 1/8


c:\Users\bened\anaconda3\envs\spacy_env\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['screenplay_input', 'year_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - accuracy: 0.0735 - loss: 751.9108 - val_accuracy: 0.0826 - val_loss: 609.5391
Epoch 2/8
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0752 - loss: 616.3588 - val_accuracy: 0.0826 - val_loss: 587.6800
Epoch 3/8
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0814 - loss: 591.7157 - val_accuracy: 0.0826 - val_loss: 565.8665
Epoch 4/8
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0882 - loss: 574.0881 - val_accuracy: 0.0826 - val_loss: 544.0271
Epoch 5/8
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.1240 - loss: 540.1219 - val_accuracy: 0.1590 - val_loss: 532.4036
Epoch 6/8
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1515 - loss: 534.4742 - val_accuracy: 0.0826 - val_loss: 514.9621
Epoch 7/8
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0798 - loss: 521.8160 - val_accuracy: 0.0826 - val_loss: 501.1057
Epoch 8/8
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0835 - loss: 493.8526 - val_accuracy: 0.1590 - val_l

In [67]:
print(type(X_test))

<class 'pandas.core.frame.DataFrame'>


In [62]:
score = ff.evaluate(X_test, y_test, verbose=1)

print("Test Loss:", score[0])
print("Test Accuracy:", score[1])

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
import matplotlib.pyplot as plt

plt.plot()

# RNN 

In [32]:
from keras.layers import LSTM, Embedding, Dense, Bidirectional, Dropout, Input, concatenate
from tensorflow.keras.models import Model

In [33]:
maxlen=10000
text_input = Input(shape=(maxlen,),name='screenplay_input')

In [34]:
embedding_layer = Embedding(len(vocab), 100, weights=[embedding_matrix], input_length=maxlen, trainable=False)(text_input)

lstm_layer = LSTM(64, return_sequences=False)(embedding_layer)

dense_1 = Dense(6, activation='relu')(lstm_layer)
dense_2 = Dense(6, activation='relu')(dense_1)

year_input = Input(shape=(1,),name='year_input')

merged = concatenate([dense_2, year_input])

output = Dense(5, activation='softmax')(merged)

model = Model(inputs=[text_input, year_input],outputs=output)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

c:\Users\bened\anaconda3\envs\spacy_env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ screenplay_input    │ (None, 10000)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 10000,     │ 18,889,400 │ screenplay_input… │
│ (Embedding)         │ 100)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     42,240 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 6)         │        390 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 6)         │         42 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ year_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 7)         │          0 │ dense_1[0][0],    │
│ (Concatenate)       │                   │            │ year_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 5)         │         40 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 18,932,112 (72.22 MB)

 Trainable params: 42,712 (166.84 KB)

 Non-trainable params: 18,889,400 (72.06 MB)

In [37]:
del df, test_vocab, train_vocab, vocab, 

In [38]:
del aus_classifications

In [41]:
model.fit(
    [X_train_padded, X_train['year']], 
    y_train,
    validation_data=([X_test_padded, X_test['year']], y_test),
    epochs=10, batch_size=64)

Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 69s 3s/step - accuracy: 0.3720 - loss: 2.0058 - val_accuracy: 0.4343 - val_loss: 1.8320
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 66s 3s/step - accuracy: 0.3913 - loss: 2.0180 - val_accuracy: 0.4404 - val_loss: 1.8539
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 67s 3s/step - accuracy: 0.3905 - loss: 1.9294 - val_accuracy: 0.3670 - val_loss: 1.8346
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 68s 3s/step - accuracy: 0.3717 - loss: 1.9945 - val_accuracy: 0.3394 - val_loss: 1.8804
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 66s 3s/step - accuracy: 0.3643 - loss: 1.9885 - val_accuracy: 0.3914 - val_loss: 1.8230
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 67s 3s/step - accuracy: 0.3887 - loss: 1.9378 - val_accuracy: 0.3609 - val_loss: 1.8359
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 67s 3s/step - accuracy: 0.3784 - loss: 1.9681 - val_accuracy: 0.3058 - val_loss: 1.8863
Epoch 8/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 67s 3s/step - accuracy: 0.3403 - loss: 2.1082 - val_accuracy: 0.3731 - val_loss:

increasing batch size seems to be effective, but epochs not so much. 

Without second dense layer 

In [ ]:
embedding_layer = Embedding(len(vocab), 100, weights=[embedding_matrix], input_length=maxlen, trainable=False)(text_input)

lstm_layer = LSTM(64, return_sequences=False)(embedding_layer)

dense_1 = Dense(6, activation='relu')(lstm_layer)
dense_2 = Dense(6, activation='relu')(dense_1)

year_input = Input(shape=(1,),name='year_input')

merged = concatenate([dense_2, year_input])

output = Dense(5, activation='softmax')(merged)

model = Model(inputs=[text_input, year_input],outputs=output)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()